In [16]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd.functional import jacobian

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
import numpy as np

In [ ]:
# We consider Net as our solution u_theta(x,t)

"""
When forming the network, we have to keep in mind the number of inputs and outputs
In ur case: #inputs = 2 (x,t)
and #outputs = 1

You can add ass many hidden layers as you want with as many neurons.
More complex the network, the more prepared it is to find complex solutions, but it also requires more data.

Let us create this network:
min 5 hidden layer with 5 neurons each.
"""


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.hidden_layer1 = nn.Linear(1, 100)
        self.output_layer = nn.Linear(100, 5)

    def forward(self, t):
        inputs = torch.cat(
            [t], axis=1
        )  # combined two arrays of 1 columns each to one array of 2 columns
        layer1_out = torch.tanh(self.hidden_layer1(inputs))
        output = self.output_layer(
            layer1_out
        )  ## For regression, no activation is used in output layer
        return output

In [43]:
def create_Phi(D, A, b):
    def Phi(Y):
        x = Y[:, :4]  # Extract first 4 columns (shape: batch_size x 4)
        u = Y[:, -1].unsqueeze(1)  # Extract last column as (batch_size x 1)

        m = torch.relu(u + torch.matmul(x, A.T) - b)  # (batch_size, 1)

        # Expand D to match batch size: (4,) -> (batch_size, 4)
        D_expanded = D.unsqueeze(0).expand(Y.shape[0], -1)  # (batch_size, 4)

        # Compute output ensuring dimensions match
        result = torch.cat((-D_expanded - m @ A, m - u), dim=1)  # (batch_size, 5)

        return result

    return Phi


# Define tensors
D = torch.tensor([-9.54, -8.16, -4.26, -11.43], dtype=torch.float32).to(device)  # (4,)
A = torch.tensor([[3.18, 2.72, 1.42, 3.81]], dtype=torch.float32).to(device)  # (1, 4)
b = torch.tensor([7.81], dtype=torch.float32).view(1, 1).to(device)  # (1, 1)

# Create function
Phi = create_Phi(D, A, b)

# Generate a batch of 500 inputs (500 rows, 5 columns)
Y = torch.rand(500, 5).to(device)

# Pass batch through function
output = Phi(Y)

# Print output shape to verify correctness
print(output.shape)  # Expected: (500, 5)

torch.Size([500, 5])


In [45]:
Y = torch.rand(500, 5).to(device)
Phi(Y)

tensor([[ 9.5400,  8.1600,  4.2600, 11.4300, -0.5198],
        [ 9.5400,  8.1600,  4.2600, 11.4300, -0.7350],
        [ 9.5400,  8.1600,  4.2600, 11.4300, -0.3240],
        ...,
        [ 9.5400,  8.1600,  4.2600, 11.4300, -0.8568],
        [ 9.5400,  8.1600,  4.2600, 11.4300, -0.9607],
        [ 9.5400,  8.1600,  4.2600, 11.4300, -0.4887]], device='cuda:0')

In [ ]:
### (2) Model
net = Net()
net = net.to(device)


def create_mse_cost_function():
    all_zeros = np.zeros((500, 1))
    pt_all_zeros = Variable(
        torch.from_numpy(all_zeros).float(), requires_grad=False
    ).to(device)
    mse_fun = torch.nn.MSELoss()  # Mean squared error

    def mse_closure(t):
        return mse_fun(t, pt_all_zeros)

    return mse_closure


mse_cost_function = create_mse_cost_function()
optimizer = torch.optim.Adam(net.parameters())

In [54]:
## PDE as loss function. Thus would use the network which we call as u_theta
def f(t, net):
    u_net = net(t)
    # the dependent variable u is given by the network based on independent variables x,t
    ## Based on our f = du/dx - 2du/dt - u, we need du/dx and du/dt
    u_t = jacobian(lambda t: (1 - torch.exp(-t)) * net(t), t, create_graph=True)[0]
    phi_u = Phi(u_net)
    pde = u_t - phi_u
    return pde

In [60]:
t_collocation = np.random.uniform(low=0.0, high=1.0, size=(500, 1))
pt_t_collocation = Variable(
    torch.from_numpy(t_collocation).float(), requires_grad=True
).to(device)
# f_out = f(pt_t_collocation, net)
net(pt_t_collocation)

tensor([[0.6519, 0.7960, 0.6131, 0.7096, 3.0064],
        [0.6332, 0.7770, 0.5433, 0.7716, 2.9799],
        [0.6355, 0.7810, 0.5609, 0.7535, 3.0058],
        ...,
        [0.6323, 0.7750, 0.5349, 0.7803, 2.9646],
        [0.6365, 0.7824, 0.5667, 0.7478, 3.0121],
        [0.6415, 0.7881, 0.5881, 0.7282, 3.0234]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [ ]:
## Data from Boundary Conditions
# u(x,0)=6e^(-3x)
## BC just gives us datapoints for training

# BC tells us that for any x in range[0,2] and time=0, the value of u is given by 6e^(-3x)
# Take say 500 random numbers of x
# t_bc = np.random.uniform(low=0.0, high=10.0, size=(500,1))
# t_bc = np.zeros((500,1))
# # compute u based on BC
# u_bc = 6*np.exp(-3*x_bc)

In [ ]:
### (3) Training / Fitting
iterations = 20000
previous_validation_loss = 99999999.0
for epoch in range(iterations):
    optimizer.zero_grad()  # to make the gradients zero

    t_collocation = np.random.uniform(low=0.0, high=10.0, size=(500, 1))
    all_zeros = np.zeros((500, 1))

    # pt_x_collocation = Variable(torch.from_numpy(x_collocation).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(
        torch.from_numpy(t_collocation).float(), requires_grad=True
    ).to(device)
    pt_all_zeros = Variable(
        torch.from_numpy(all_zeros).float(), requires_grad=False
    ).to(device)

    f_out = f(pt_t_collocation, net)  # output of f(x,t)
    mse_f = mse_cost_function(f_out)

    # Combining the loss functions
    loss = mse_f

    loss.backward()  # This is for computing gradients using backward propagation
    optimizer.step()  # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta

    with torch.autograd.no_grad():
        print(epoch, "Traning Loss:", loss.data)

0 Traning Loss: tensor(10.6953, device='cuda:0')
1 Traning Loss: tensor(15.1207, device='cuda:0')
2 Traning Loss: tensor(18.1619, device='cuda:0')
3 Traning Loss: tensor(19.0968, device='cuda:0')
4 Traning Loss: tensor(18.8548, device='cuda:0')
5 Traning Loss: tensor(15.0831, device='cuda:0')
6 Traning Loss: tensor(12.5414, device='cuda:0')
7 Traning Loss: tensor(9.1606, device='cuda:0')
8 Traning Loss: tensor(6.2325, device='cuda:0')
9 Traning Loss: tensor(5.1097, device='cuda:0')
10 Traning Loss: tensor(4.4931, device='cuda:0')
11 Traning Loss: tensor(4.9284, device='cuda:0')
12 Traning Loss: tensor(5.3571, device='cuda:0')
13 Traning Loss: tensor(6.6948, device='cuda:0')
14 Traning Loss: tensor(6.8434, device='cuda:0')
15 Traning Loss: tensor(5.8791, device='cuda:0')
16 Traning Loss: tensor(5.4301, device='cuda:0')
17 Traning Loss: tensor(3.9302, device='cuda:0')
18 Traning Loss: tensor(2.8871, device='cuda:0')
19 Traning Loss: tensor(2.0061, device='cuda:0')
20 Traning Loss: tensor

KeyboardInterrupt: 

In [59]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

fig = plt.figure()
# ax = fig.gca(projection="3d")

# x=np.arange(0,2,0.02)
t = np.arange(0, 10, 0.02)
# ms_x, ms_t = np.meshgrid(x, t)
# ## Just because meshgrid is used, we need to do the following adjustment
# x = np.ravel(ms_x).reshape(-1,1)
# t = np.ravel(ms_t).reshape(-1,1)

# pt_x = Variable(torch.from_numpy(x).float(), requires_grad=True).to(device)
pt_t = Variable(torch.from_numpy(t).float(), requires_grad=True).to(device)
pt_u = net(pt_t)
# u=pt_u.data.cpu().numpy()
# ms_u = u.reshape(ms_x.shape)

# surf = ax.plot_surface(ms_x,ms_t,ms_u, cmap=cm.coolwarm,linewidth=0, antialiased=False)


# ax.zaxis.set_major_locator(LinearLocator(10))
# ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# fig.colorbar(surf, shrink=0.5, aspect=5)

# plt.show()

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

<Figure size 640x480 with 0 Axes>